In [ ]:
import polars as pl

In [ ]:
scan_all = pl.scan_parquet("output2.parquet") # 원본 유지
scan_preprocess = pl.scan_parquet("output2.parquet") # 전처리 결과 저장

# 중복값 확인

In [ ]:
def key_uniqueness_query(lf: pl.LazyFrame, key_col: str):
    """
    lf: LazyFrame (scan_parquet 결과)
    key_col: 고유해야 한다고 보는 컬럼 이름 (예: 'mdr_report_key')

    반환: total_rows, unique_keys 두 개 컬럼을 가진 LazyFrame 쿼리
    """
    q = lf.select(
        [
            pl.count().alias("total_rows"),
            pl.col(key_col).n_unique().alias("unique_keys"),
        ]
    )
    return q

In [ ]:
key_uniqueness_query(scan_preprocess, "mdr_report_key").collect()

# categorical 변환

## 단일 컬럼

### 변환하기 전 고윳값 확인

In [ ]:
def profile_query(lf: pl.LazyFrame, col: str, top_n: int = 10):
    """
    lf: LazyFrame
    col: 분석할 컬럼
    top_n: 상위 몇 개 카테고리 보여줄지
    """

    # 전체 행 수
    total_rows_q = lf.select(pl.count().alias("total_rows"))

    # 고윳값 개수
    unique_values_q = lf.select(pl.col(col).n_unique().alias("unique_values"))

    # value_counts
    value_counts_q = (
        lf
        .group_by(col)
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .head(top_n)
    )

    # 세 정보(total_rows, unique_values, value_counts)를 함께 조합
    q = (
        value_counts_q
        .join(total_rows_q, how="cross")
        .join(unique_values_q, how="cross")
        .with_columns([
            (pl.col("count") / pl.col("total_rows")).alias("value_ratio"),
            (pl.col("unique_values") / pl.col("total_rows")).alias("unique_ratio")
        ])
        .select([col, "count", "value_ratio", "unique_ratio"])
    )

    return q


event_location

In [ ]:
profile_query(
    scan_preprocess,
    "event_location",
    top_n=10
).collect()

device_0_openfda_device_class

In [ ]:
profile_query(
    scan_preprocess,
    "device_0_openfda_device_class",
    top_n=10
).collect()

In [ ]:
# CATEGORICAL 타입 변환 함수
def cast_to_categorical(lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    단일 컬럼을 Categorical로 변환
    """
    if col not in lf.columns:
        print(f"[WARN] Column '{col}' not found. Skipped.")
        return lf


    return lf.with_columns(
        pl.col(col).cast(pl.Categorical)
    )


In [ ]:
scan_preprocess = cast_to_categorical(scan_preprocess, "event_location")

In [ ]:
scan_preprocess.collect_schema()["event_location"]


## _openfda_device_class'로 끝나는 모든 컬럼을 Categorical로 변환

### 변환하기 전 고윳값 확인(위에서 했음)

In [ ]:
schema_cols = scan_preprocess.collect_schema().names()

openfda_class_cols = [c for c in schema_cols if c.endswith("_openfda_device_class")]
openfda_class_cols

### 함수

In [ ]:
import polars as pl

def cast_device_openfda_classes_to_cat(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    이름이 '*_openfda_device_class'로 끝나는 모든 컬럼을 Categorical로 변환
    예: device_0_openfda_device_class, device_1_openfda_device_class, ...
    """
    # 1) 컬럼 이름 중에서 패턴 맞는 것만 골라오기
    target_cols = [
        col for col in lf.columns
        if col.endswith("_openfda_device_class")
    ]

    if not target_cols:
        return lf  # 해당 컬럼이 없으면 그대로 반환

    # 2) 골라낸 컬럼들을 전부 Categorical로 캐스팅
    exprs = [pl.col(col).cast(pl.Categorical) for col in target_cols]

    return lf.with_columns(exprs)

In [ ]:
scan_preprocess = cast_device_openfda_classes_to_cat(scan_preprocess)


# boolean으로 변환

## 변환하기 전에 확인

In [ ]:
profile_query(
    scan_preprocess,
    "adverse_event_flag",
    top_n=10
).collect()

In [ ]:
profile_query(
    scan_preprocess,
    "report_to_fda",
    top_n=10
).collect()

In [ ]:
profile_query(
    scan_preprocess,
    "manufacturer_link_flag",
    top_n=10
).collect().to_pandas()

In [ ]:
profile_query(
    scan_preprocess,
    "product_problem_flag",
    top_n=10
).collect().to_pandas()

In [ ]:
def cast_flags_to_bool(
    lf: pl.LazyFrame,
    flag_cols: list[str] = ["adverse_event_flag", "manufacturer_link_flag", "product_problem_flag", "report_to_fda"],
) -> pl.LazyFrame:
    return lf.with_columns([
        pl.when(pl.col(col) == "Y")
          .then(pl.lit(True))
          .when(pl.col(col) == "N")
          .then(pl.lit(False))
          .otherwise(None)
          .alias(col)
        for col in flag_cols
        if col in lf.columns
    ])

In [ ]:
scan_preprocess = cast_flags_to_bool(scan_preprocess)

In [ ]:
{c: scan_preprocess.schema[c] for c in [
    "adverse_event_flag",
    "manufacturer_link_flag",
    "product_problem_flag",
    "report_to_fda"
]}

In [ ]:
profile_query(
    scan_preprocess,
    "product_problem_flag",
    top_n=10
).collect()

# 날짜 컬럼 date 타입 변환

## 변환하기 전 확인

길이가 8자리가 맞는지, 다른 포맷 섞여있는지, 빈문자열이 null이외에 unk이런 거 있는지 확인하는 함수

In [ ]:
def summarize_date_format(lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    날짜 컬럼 원시 포맷 요약:
    - 문자열 길이 (str_len)
    - 8자리 숫자(YYYYMMDD) 여부 (is_8_digit)
    - '-' / '/' 포함 여부 (has_dash, has_slash)
    """
    if col not in lf.columns:
        raise ValueError(f"컬럼 '{col}'이(가) lf에 없음")

    return (
        lf
        .select(
            pl.col(col)
              .cast(pl.Utf8)
              .alias(col)
        )
        .with_columns([
            pl.col(col).str.len_chars().alias("str_len"),
            pl.col(col).str.contains(r"^\d{8}$").alias("is_8_digit"),
            pl.col(col).str.contains("-").alias("has_dash"),
            pl.col(col).str.contains("/").alias("has_slash"),
        ])
        .group_by(["str_len", "is_8_digit", "has_dash", "has_slash"])
        .len()
        .rename({"len": "n_rows"})        # 개수 컬럼 이름 변경
        .sort("n_rows", descending=True)  # 많이 나온 조합부터 보기
    )


In [ ]:
summarize_date_format(scan_preprocess, "date_of_event").collect() # 널값 깔끔 나머지도 불순물 없음
#summarize_date_format(scan_preprocess, "date_received").collect() # 널값 없음, 포맷 깔끔
#summarize_date_format(scan_preprocess, "device_date_of_manufacturer").collect() # 포맷 깔끔

date 타입 변환 함수

def cast_date_cols(
    lf: pl.LazyFrame,
    date_cols: list[str],
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    """
    YYYYMMDD 형식의 날짜 컬럼들을 pl.Date로 변환하는 함수.
    - strict=False: 파싱 실패 값은 null로 떨어짐.
    """
    exprs = []
    for col in date_cols:
        if col in lf.collect_schema().names():
            exprs.append

In [ ]:
import polars as pl

def cast_date_cols_safe(
    lf: pl.LazyFrame,
    date_cols: list[str],
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    schema = lf.collect_schema()
    exprs = []

    for col in date_cols:
        if col not in schema:
            continue

        dt = schema[col]

        # 이미 Date면 건드리지 않음
        if dt == pl.Date:
            continue

        # Datetime이면 Date로만 다운캐스트
        if dt == pl.Datetime:
            exprs.append(pl.col(col).cast(pl.Date, strict=False).alias(col))
            continue

        # 그 외(Utf8/Int 등)만 fmt로 파싱
        exprs.append(
            pl.col(col)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(col)
        )

    return lf if not exprs else lf.with_columns(exprs)


In [ ]:
date_cols = [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer",
]

scan_preprocess = cast_date_cols_safe(scan_preprocess, date_cols=date_cols)


In [ ]:
{c: scan_preprocess.schema[c] for c in [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer"
]}

In [ ]:
scan_preprocess.select([
    pl.len().alias("n_rows"),
    pl.col("date_of_event").is_null().sum().alias("n_null_event"),
    pl.col("date_of_event").drop_nulls().head(5).alias("sample"),
]).collect()


# device_age_text 단위 맞추기

In [ ]:
(
    scan_preprocess
    .select("device_0_device_age_text")
    .limit(20)
).collect().to_pandas()

In [ ]:
# null값 확인
scan_preprocess.select(pl.col("device_0_device_age_text").null_count()).collect()

In [ ]:
# 단위 형태 확인
col = "device_0_device_age_text"

units = (
    scan_preprocess
    .select([
        pl.col(col)
          .cast(pl.Utf8)
          .str.to_lowercase()
          .alias("age_text_lower")
    ])
    .with_columns([
        # 예: "3 days" → "days", "2 wk" → "wk"
        pl.col("age_text_lower")
          .str.extract(r"\d+\s*([a-zA-Z]+)", group_index=1)
          .alias("unit_token")
    ])
    .filter(pl.col("unit_token").is_not_null())
    .group_by("unit_token")
    .len()
    .sort("len", descending=True)
    .limit(50)   # 상위 50개만
)

units.collect()

In [ ]:
def normalize_device_age_to_days(
    lf: pl.LazyFrame,
    src_col: str = "device_0_device_age_text",
    dst_col: str = "device_0_device_age_days",
) -> pl.LazyFrame:
    return (
        lf.with_columns([
            pl.col(src_col).cast(pl.Utf8).str.to_lowercase().alias("_age_text_lower"),
        ])
        .with_columns([
            # 숫자 추출
            pl.col("_age_text_lower")
              .str.extract(r"(\d+(\.\d+)?)", group_index=1)
              .cast(pl.Float64)
              .alias("_age_value"),

            # 단위 추출
            pl.col("_age_text_lower")
              .str.extract(r"(day|da|days|d|week|weeks|wk|wks|month|months|mo|year|years|yr|yrs)", group_index=1)
              .alias("_age_unit"),
        ])
        .with_columns([
            pl.when(pl.col("_age_value").is_null())
              .then(None)  # 숫자 자체가 없으면 null
            .when(pl.col("_age_unit").is_null())
              .then(pl.col("_age_value"))  # 단위 없으면 day로 가정
            .when(pl.col("_age_unit").str.contains("day|days|d"))
              .then(pl.col("_age_value"))
            .when(pl.col("_age_unit").str.contains("week|weeks|wk|wks"))
              .then(pl.col("_age_value") * 7)
            .when(pl.col("_age_unit").str.contains("month|months|mo"))
              .then(pl.col("_age_value") * 30)
            .when(pl.col("_age_unit").str.contains("year|years|yr|yrs"))
              .then(pl.col("_age_value") * 365)
            .otherwise(None)
            .cast(pl.Int64)
            .alias(dst_col)
        ])
        .drop(["_age_text_lower", "_age_value", "_age_unit"])
    )


"age_days" 컬럼 새로 만들어서 단위 변환한 결과 저장

In [ ]:
def normalize_all_device_age_cols_to_days(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    LazyFrame 내에서 이름이 '*_device_age_text'로 끝나는 모든 컬럼을 탐색하여,
    각 컬럼에 대응되는 '*_device_age_days' 컬럼을 생성한다.

    - 각 *_device_age_text 컬럼은 자유 텍스트 형태의 기기 사용 기간 정보
      (예: '3 days', '2 weeks', '1 yr', '10')를 담고 있다고 가정한다.
    - 내부적으로 normalize_device_age_to_days()를 호출하여
      숫자 + 단위(day/week/month/year)를 day 단위 정수(Int64)로 정규화한다.
    - 단위가 명시되지 않은 경우 day로 간주한다.
    - 숫자 추출이 불가능한 경우 결과는 null이 된다.
    - 이미 존재하는 '*_device_age_days' 컬럼이 있더라도 덮어써진다.

    Parameters
    ----------
    lf : pl.LazyFrame
        MAUDE 전처리 파이프라인 상의 입력 LazyFrame.

    Returns
    -------
    pl.LazyFrame
        각 *_device_age_text 컬럼에 대해 대응되는 *_device_age_days 컬럼이
        추가된 LazyFrame.
    """
    schema_cols = lf.collect_schema().names()

    age_text_cols = [c for c in schema_cols if c.endswith("_device_age_text")]

    for src_col in age_text_cols:
        dst_col = src_col.replace("_device_age_text", "_device_age_days")
        lf = normalize_device_age_to_days(lf, src_col=src_col, dst_col=dst_col)

    return lf


In [ ]:
scan_preprocess = scan_preprocess.pipe(normalize_all_device_age_cols_to_days)

# product_problem_flag × event_type
* product_problem_flag 보고서가 기기의 품질, 성능 또는 안전성과 관련된지 여부
* event_type 이상 사건과 관련된 결과 

In [ ]:
{c: scan_preprocess.schema[c] for c in [
    "product_problem_flag",
    "event_type"
]}

### event_type 종류 확인

In [ ]:
# event_type 확인
openfda_vc = (
    scan_preprocess
    .group_by("event_type")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

In [ ]:
scan_preprocess = cast_to_categorical(scan_preprocess, "event_type")

event_type == "Malfunction"이면
product_problem_flag=True 이어야 맞지 않을까?

In [ ]:
(
    scan_preprocess
    .filter(
        (pl.col("event_type") == "Malfunction")
        & (pl.col("product_problem_flag") == False)
    )
    .select(pl.len().alias("n_inconsistent"))
).collect()
# 4241값이 나와야 하는데 event_type categoricla로 바꾸고 나서 하면 0 나와요 자꾸..

In [ ]:
(
    scan_preprocess
    .filter(
        (pl.col("event_type") == "Malfunction")
        & (pl.col("product_problem_flag") == False)
    )
    .select([
        "event_type",
        "product_problem_flag",
        "product_problems",
        "report_source_code",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(1000)
).collect().to_pandas()

- `event_type = "Malfunction"`
    
    → 보고서 상으로는 “기기가 의도대로 작동하지 않았다”로 분류
    
- `product_problem_flag = "N"`
    
    → 그런데 **그 오작동이 재발해도 사망/중대한 부상 위험이 크지 않다고 판단했거나**,
    
    아예 “결함/오작동 때문은 아니다” 쪽으로 코딩한 것일 수 있음
    

이상한 패턴 아니라고 판단

# adverse_event_flag event_type과 비교하여 결측치 처리
* adverse_event_flag 의료기기 사용이 환자에게 이상 사례를 초래한 것으로 의심되는 사건의 여부

In [ ]:
# 이상 케이스 수 확인
weird_count = (
    scan_preprocess
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & (pl.col("adverse_event_flag") == False)
    )
    .select(pl.len().alias("n_weird"))
)

weird_count.collect()

In [ ]:
# 뜯어보기
weird_sample = (
    scan_preprocess
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & (pl.col("adverse_event_flag") == False)
    )
    .select([
        "event_type",
        "adverse_event_flag",
        "report_source_code",
        "date_of_event",
        "date_received",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(20)
)

weird_sample.collect().to_pandas()

In [ ]:
# 결측치 케이스: Death/Injury 인데 flag가 null
weird_null_count = (
    scan_preprocess
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & pl.col("adverse_event_flag").is_null()
    )
    .select(pl.len().alias("n_null"))
)

weird_null_count.collect()

weird_null_sample = (
    scan_preprocess
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & pl.col("adverse_event_flag").is_null()
    )
    .select([
        "mdr_report_key",
        "event_type",
        "adverse_event_flag",
        "report_source_code",
        "date_of_event",
        "date_received",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(20)
)

weird_null_sample.collect()

 결측치 케이스: Death/Injury 인데 flag가 null인 결과 없음

In [ ]:
# null값인 애들 event_type 유형 확인
(
    scan_preprocess
    .filter(pl.col("adverse_event_flag").is_null())
    .select([
        "mdr_report_key",
        "event_type",
        "adverse_event_flag",
        "report_source_code",
        "date_of_event",
        "date_received",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(90)
).collect().to_pandas()

### 결론

78건null —> no로 채운다

- 논리적으로 맞춰서 새로운 파생변수 맞춘다.
    - event_type에 맞춰서 flag 수정

# 날짜 컬럼들 순서, 범위가 이상한애들 샘플링 해서 확인 및 처리

In [ ]:
date_gaps_all = (
    scan_preprocess
    .select(["date_of_event", "date_received", "device_date_of_manufacturer"])
    .drop_nulls()
    .with_columns([
        (pl.col("date_of_event") - pl.col("device_date_of_manufacturer"))
            .dt.total_days()
            .alias("days_event_minus_mfr"),

        (pl.col("date_received") - pl.col("date_of_event"))
            .dt.total_days()
            .alias("days_recv_minus_event"),

        (pl.col("date_received") - pl.col("device_date_of_manufacturer"))
            .dt.total_days()
            .alias("days_recv_minus_mfr"),
    ])
)


In [ ]:
date_order_violations = (
    date_gaps_all
    .select([
        pl.len().alias("n_rows"),

        # 사건이 제조보다 전에 일어난 것 (말이 안 됨)
        (pl.col("days_event_minus_mfr") < 0).sum().alias("n_event_before_mfr"),

        # FDA가 사건보다 먼저 받은 것 (말이 안 됨)
        (pl.col("days_recv_minus_event") < 0).sum().alias("n_recv_before_event"),

        # FDA가 제조일보다 먼저 받은 것 (이것도 말이 안 됨)
        (pl.col("days_recv_minus_mfr") < 0).sum().alias("n_recv_before_mfr"),
    ])
    .with_columns([
        (pl.col("n_event_before_mfr") / pl.col("n_rows")).alias("ratio_event_before_mfr"),
        (pl.col("n_recv_before_event") / pl.col("n_rows")).alias("ratio_recv_before_event"),
        (pl.col("n_recv_before_mfr") / pl.col("n_rows")).alias("ratio_recv_before_mfr"),
    ])
)

date_order_violations.collect()

- 사건이 제조보다 먼저: 6,611
- 사건보다 보고가 먼저: 1,087
- 제조보다 보고가 먼저: 382

### 실제 사례 확인

In [ ]:
cols_to_view = [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer"
]

flagged = (
    date_gaps_all
    .with_columns([
        (pl.col("days_event_minus_mfr") < 0).alias("viol_event_before_mfr"),
        (pl.col("days_recv_minus_event") < 0).alias("viol_recv_before_event"),
        (pl.col("days_recv_minus_mfr") < 0).alias("viol_recv_before_mfr"),
    ])
    .with_columns([
        pl.any_horizontal([
            pl.col("viol_event_before_mfr"),
            pl.col("viol_recv_before_event"),
            pl.col("viol_recv_before_mfr"),
        ]).alias("any_date_violation")
    ])
)

In [ ]:
# 사건 발생일이 기기 제조일 보다 먼저
df_event_before_mfr = (
    flagged
    .filter(pl.col("viol_event_before_mfr"))
    .select(cols_to_view)
    .collect()
)

# 보고 받은 날짜가 사건 발생일보다 이전
df_recv_before_event = (
    flagged
    .filter(pl.col("viol_recv_before_event"))
    .select(cols_to_view)
    .collect()
)

# 제조일보다 fda 보고가 이전 
df_recv_before_mfr = (
    flagged
    .filter(pl.col("viol_recv_before_mfr"))
    .select(cols_to_view)
    .collect()
)

df_any_violation = (
    flagged
    .filter(pl.col("any_date_violation"))
    .select(cols_to_view)
    .collect()
)

In [ ]:
# 사건 발생일이 제조보다 먼저 
df_event_before_mfr.to_pandas()

In [ ]:
# 사건보다 보고가 먼저
df_recv_before_event.to_pandas()

In [ ]:
# 제조보다 보고가 먼저 
df_recv_before_mfr.to_pandas()

말도 안되는 날짜 확인

In [ ]:
# 말도 안되는 날짜 
lf = scan_preprocess
from datetime import date

MIN_DATE = date(1800, 1, 1)
MAX_DATE = date(2100, 12, 31)

too_old_future = (
    lf.select([
        pl.len().alias("n_rows"),

        (pl.col("date_of_event") < pl.lit(MIN_DATE)).sum().alias("n_event_too_old"),
        (pl.col("date_received") < pl.lit(MIN_DATE)).sum().alias("n_recv_too_old"),
        (pl.col("device_date_of_manufacturer") < pl.lit(MIN_DATE)).sum().alias("n_mfr_too_old"),

        (pl.col("date_of_event") > pl.lit(MAX_DATE)).sum().alias("n_event_too_future"),
        (pl.col("date_received") > pl.lit(MAX_DATE)).sum().alias("n_recv_too_future"),
        (pl.col("device_date_of_manufacturer") > pl.lit(MAX_DATE)).sum().alias("n_mfr_too_future"),
    ])
    .collect()
)

too_old_future


없음

1900년대 날짜 확인

In [ ]:
# 몇 개 있는지 확인
lf = scan_preprocess

cnt_1900s = (
    lf.select([
        pl.len().alias("n_rows"),

        # date_of_event
        ((pl.col("date_of_event").dt.year() >= 1900) & (pl.col("date_of_event").dt.year() <= 1999))
          .sum().alias("n_event_1900s"),

        # date_received
        ((pl.col("date_received").dt.year() >= 1900) & (pl.col("date_received").dt.year() <= 1999))
          .sum().alias("n_recv_1900s"),

        # device_date_of_manufacturer
        ((pl.col("device_date_of_manufacturer").dt.year() >= 1900) & (pl.col("device_date_of_manufacturer").dt.year() <= 1999))
          .sum().alias("n_mfr_1900s"),
    ])
    .with_columns([
        (pl.col("n_event_1900s") / pl.col("n_rows")).alias("ratio_event_1900s"),
        (pl.col("n_recv_1900s") / pl.col("n_rows")).alias("ratio_recv_1900s"),
        (pl.col("n_mfr_1900s") / pl.col("n_rows")).alias("ratio_mfr_1900s"),
    ])
    .collect()
)

cnt_1900s


In [ ]:
# 실제 사례 확인
df_event_1900s = (
    scan_preprocess
    .filter(
        (pl.col("date_of_event").is_not_null()) &
        (pl.col("date_of_event").dt.year() >= 1900) &
        (pl.col("date_of_event").dt.year() <= 1999)
    )
    .select([
        "date_of_event",
        "date_received",
        "device_date_of_manufacturer",
        # 보고 싶으면 추가(존재하는 컬럼만)
        # "mdr_report_key",
        # "event_type",
        # "adverse_event_flag",
        # "product_problem_flag",
    ])
    .collect().to_pandas()
)

df_event_1900s

이 경우에 date_received는 다 2024연도가 맞는데

manufacturer값이 널값이 많고 아주 예전인 경우도 있음

In [ ]:
date_gaps_all = (
    scan_preprocess
    .select([
        "date_of_event",
        "date_received",
        "device_date_of_manufacturer",
    ])
    .drop_nulls()  # 세 날짜 다 있는 행만 대상으로 순서/차이 확인
    .with_columns([
        pl.col("date_of_event")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .alias("date_of_event"),

        pl.col("date_received")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .alias("date_received"),

        pl.col("device_date_of_manufacturer")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .alias("device_date_of_manufacturer"),
    ])
    .with_columns([
        # 사건일 - 제조일 (정상이라면 >= 0이어야 함)
        (pl.col("date_of_event").cast(pl.Int64) - pl.col("device_date_of_manufacturer").cast(pl.Int64))
            .alias("days_event_minus_mfr"),

        # 수신일 - 사건일 (정상이라면 >= 0)
        (pl.col("date_received").cast(pl.Int64) - pl.col("date_of_event").cast(pl.Int64))
            .alias("days_recv_minus_event"),

        # 수신일 - 제조일 (정상이라면 >= 0)
        (pl.col("date_received").cast(pl.Int64) - pl.col("device_date_of_manufacturer").cast(pl.Int64))
            .alias("days_recv_minus_mfr"),
    ])
)


이상값(1900년대)이 특정 기간, 기기, 특정 product_code에 있는지 보기

In [ ]:
flagged_bad = (
    lf.with_columns([
        (
            pl.col("date_of_event").is_not_null()
            & (pl.col("date_of_event").dt.year() >= 1900)
            & (pl.col("date_of_event").dt.year() <= 1999)
        ).alias("is_bad_event_1900s")
    ])
)


In [ ]:
# 특정 기간에 몰려있는지
bad_by_recv_month = (
    flagged_bad
    .with_columns(pl.col("date_received").dt.truncate("1mo").alias("recv_month"))
    .group_by("recv_month")
    .agg([
        pl.len().alias("n_total"),
        pl.col("is_bad_event_1900s").sum().alias("n_bad"),
        pl.col("is_bad_event_1900s").mean().alias("ratio_bad"),
    ])
    .sort("recv_month")
    .collect()
)

bad_by_recv_month.to_pandas()

In [ ]:
# 특정 product_code에 몰려있는지
bad_by_product_code = (
    flagged_bad
    .group_by("device_0_device_report_product_code")
    .agg([
        pl.len().alias("n_total"),
        pl.col("is_bad_event_1900s").sum().alias("n_bad"),
        pl.col("is_bad_event_1900s").mean().alias("ratio_bad"),
    ])
    .filter(pl.col("n_total") >= 50)  # 표본 너무 적은 건 제외(원하면 조정)
    .sort("ratio_bad", descending=True)
    .collect()
)

bad_by_product_code.to_pandas()


In [ ]:
# 특정 제조사에 몰려있는지
bad_by_mfr = (
    flagged_bad
    .group_by("device_0_manufacturer_d_name")
    .agg([
        pl.len().alias("n_total"),
        pl.col("is_bad_event_1900s").sum().alias("n_bad"),
        pl.col("is_bad_event_1900s").mean().alias("ratio_bad"),
    ])
    .filter(pl.col("n_total") >= 50)
    .sort("ratio_bad", descending=True)
    .collect()
)

bad_by_mfr.to_pandas()

1990년대 event가 기기별, 제조사별, 특정 기간별 어디에 제일 많이 몰려있는지 봤을 때 어디 하나에 특정하게 많이 몰려있다는 것은 찾지 못함

결론

- event가 1900년대이면 date_of_event를 null값으로 채우자.
- date_of_event 기준을 어떻게…
    - 어느 기간까지를 보고 지연으로 보는가를 따져보고, 보고지연 기간이 넘어가면 이상치 판단하여 제거
    - (ex. 데이터 연도 - 1까지는 살리자)
- received가 다 2024인지 확인하기(이게 맞으니까)
    - 2024년이 아니면 2024로 날짜를 바꾸거나, 삭제
    - 그럼 1년치씩 따로 전처리하고 합치자!

- 중복은 5년치 합치고 나서 돌리자

## report_to_fda 'I'를 null로 처리

In [ ]:
# report_to_fda 컬럼에서 "I" 값을 결측치(null)로 치환
lf = scan_preprocess

lf = lf.with_columns(
    pl.when(
        # 1) 문자열로 캐스팅 후 공백 제거(앞/뒤)해서 "I"인지 확인
        #    -> " I", "I " 같은 변형도 잡기 위한 방어 로직
        pl.col("report_to_fda").cast(pl.Utf8).str.strip_chars() == "I"
    )
    .then(
        # 2) 조건에 해당하면 null로 변경
        None
    )
    .otherwise(
        # 3) 그 외 값은 원래 값을 유지
        pl.col("report_to_fda")
    )
    # 4) 같은 컬럼명으로 alias 해서 원본 컬럼을 덮어씀
    .alias("report_to_fda")
)


In [ ]:
after_vc = (
    lf.with_columns(
        pl.when(pl.col("report_to_fda") == "I")
          .then(None)
          .otherwise(pl.col("report_to_fda"))
          .alias("report_to_fda")
    )
    .select(pl.col("report_to_fda").value_counts(sort=True))
    .unnest("report_to_fda")
    .sort("count", descending=True)
    .collect()
)

after_vc


# device_class 1,2,3 제외 null로 처리

In [ ]:
scan_preprocess.collect_schema()["device_0_openfda_device_class"]


In [ ]:
def nullify_non_123_openfda_device_class(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    이름이 '*_openfda_device_class'로 끝나는 모든 컬럼을 찾아,
    값이 "1", "2", "3"인 경우만 유지하고 나머지는 null로 치환한다.

    - 문자열 양끝 공백은 제거(strip)해서 비교한다.
    - 원본이 categorical이어도 비교는 문자열로 안전하게 처리한다.
    - 컬럼이 없으면 입력 LazyFrame을 그대로 반환한다.
    """
    schema_cols = lf.collect_schema().names()
    target_cols = [c for c in schema_cols if c.endswith("_openfda_device_class")]

    if not target_cols:
        return lf

    allowed = ["1", "2", "3"]

    exprs = []
    for c in target_cols:
        cleaned = pl.col(c).cast(pl.Utf8).str.strip_chars()

        exprs.append(
            pl.when(cleaned.is_in(allowed))
              .then(cleaned)          # "1","2","3"만 남김
              .otherwise(None)        # 나머지는 null
              .alias(c)               # 원본 컬럼 덮어쓰기
        )

    return lf.with_columns(exprs)

In [ ]:
scan_preprocess = nullify_non_123_openfda_device_class(scan_preprocess)

In [ ]:
scan_preprocess = cast_device_openfda_classes_to_cat(scan_preprocess)

In [ ]:
col = "device_0_openfda_device_class"

after_vc = (
    scan_preprocess
    .select(pl.col(col).value_counts(sort=True))
    .unnest(col)
    .sort("count", descending=True)
    .collect()
)

after_vc


In [ ]:
scan_preprocess.collect_schema()["device_0_openfda_device_class"]

# event_type에 맞춰서 adverse_event_flag 수정한 새로운 파생변수 만들기

In [ ]:
lf = lf.with_columns(
    pl.when(pl.col("event_type").is_in(["Death", "Injury"]))
      .then(True)
      .otherwise(False)
      .alias("adverse_event_flag_logic")
)


In [ ]:
# 새 컬럼 이름 "adverse_event_flag_logic"

cols = ["event_type", "adverse_event_flag", "adverse_event_flag_logic"]

# 1) True 샘플 (Death/Injury)
true_df = (
    lf.filter(pl.col("adverse_event_flag_logic") == True)
      .select(cols)
      .collect()
)
true_samples = true_df.sample(n=min(20, true_df.height), seed=42)

# 2) False 샘플 (Death/Injury 제외 전부)
false_df = (
    lf.filter(pl.col("adverse_event_flag_logic") == False)
      .select(cols)
      .collect()
)
false_samples = false_df.sample(n=min(20, false_df.height), seed=42)

# 3) 충돌 샘플 (원본 flag가 있고, 논리 파생값과 다른 경우)
conflict_df = (
    lf.filter(
        pl.col("adverse_event_flag").is_not_null()
        & (pl.col("adverse_event_flag") != pl.col("adverse_event_flag_logic"))
    )
    .select(cols)
    .collect()
)
conflict_samples = conflict_df.sample(n=min(20, conflict_df.height), seed=42)

true_samples, false_samples.to_pandas(), conflict_samples.to_pandas()


# date_of_event가 1900년대이면 null로 처리

In [ ]:
scan_preprocess.collect_schema()["date_of_event"]

In [ ]:
import polars as pl

# 1) clean 컬럼 생성 (dtype 고정)
scan_preprocess = scan_preprocess.with_columns(
    pl.when(pl.col("date_of_event").dt.year().is_between(1900, 1999))
      .then(pl.lit(None, dtype=pl.Date))          # <- 핵심: Date로 null 지정
      .otherwise(pl.col("date_of_event"))        
      .alias("date_of_event_clean")
)


In [ ]:
scan_preprocess = cast_date_cols_safe(
    scan_preprocess,
    date_cols=["date_of_event_clean",
               "date_of_event"]
)

In [ ]:
scan_preprocess.select(pl.col("date_of_event_clean")).dtypes

In [ ]:
scan_preprocess.select(pl.col("date_of_event")).dtypes

In [ ]:
scan_preprocess.select(
    pl.col("date_of_event").is_null().sum().alias("n_null")
).collect()


In [ ]:

# 2) 변환 전/후 null 비교 + (검증용) 1900s 카운트
check = scan_preprocess.select([
    pl.len().alias("n_rows"),

    # 변환 전: 원본 null
    pl.col("date_of_event").is_null().sum().alias("n_null_orig"),

    # 원본에서 1900년대였던 행 수(= clean으로 null 만든 타겟)
    #pl.col("date_of_event").dt.year().is_between(1900, 1999).sum().alias("n_1900s_orig"),

    # 변환 후: clean null
    pl.col("date_of_event_clean").is_null().sum().alias("n_null_clean")
]).collect()

check


In [ ]:
from pathlib import Path
Path("data").mkdir(parents=True, exist_ok=True)

out_path = "data/scan_preprocess.parquet"
scan_preprocess.sink_parquet(out_path)